In [ ]:
!pip install duckdb

StatementMeta(, , , Waiting, )

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.5/18.5 MB 129.2 MB/s eta 0:00:00


In [ ]:
import duckdb
df = duckdb.sql("""
install httpfs;
load httpfs;
install spatial;
load spatial;
SET s3_region='us-west-2';
select
names['common'][1][1]['value'][1] as name,
categories['main'] as category,
ST_AsText(ST_GeomFromWKB(geometry)) AS wkt,
ST_X(ST_GeomFromWKB(geometry)) AS X,
ST_Y(ST_GeomFromWKB(geometry))  AS Y,
addresses[1]['country'][1] as country
from read_parquet("s3://overturemaps-us-west-2/release/2023-07-26-alpha.0/theme=places/type=*/*", filename=true, hive_partitioning=1)
where addresses[1]['country'][1] = 'AU'
           """).df()
df =spark.createDataFrame(df)
df.write.mode("append").format("delta").saveAsTable("overturemaps")

StatementMeta(, , , Waiting, )

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

StatementMeta(, , -1, Waiting, )

_**Create dimension table**_

In [ ]:
df = spark.sql('select distinct category as category from overturemaps where category is not null')
df.write.mode("overwrite").format("delta").saveAsTable("category")

StatementMeta(, , , Waiting, )